In [ ]:
!pip install groq
import os
from google.colab import drive
from groq import Groq
from time import sleep

drive.mount('/content/drive')

API_KEY = "siia läheb api võti"

In [ ]:
def read_file_into_memory(filepath):
  try:
    with open(filepath, 'r') as file:
      content = file.read()
      #content = content.replace('\n', '')
      return content
  except FileNotFoundError:
    print(f"Error: File '{filepath}' not found.")
    return None


In [ ]:
def tekita_treening_andmestik():
  artiklid = []
  kokkuvotted = []
  for kaust in ["uudised", "arvamused"]:
    for i in range(10):
      artiklid.append(read_file_into_memory(f"/content/drive/MyDrive/loputoo/arenduskorpus/{kaust}/{kaust[:-2]}{i+1}/{kaust[:-2]}{i+1}_alg.txt"))
      kokkuvotted.append(read_file_into_memory(f"/content/drive/MyDrive/loputoo/arenduskorpus/{kaust}/{kaust[:-2]}{i+1}/{kaust[:-2]}{i+1}.txt"))
  return artiklid, kokkuvotted

artiklid, kokkuvote = tekita_treening_andmestik()

In [ ]:
def tekita_viipa_ajalugu():
  ajalugu = [{"role": "system",
    "content": "Sa oled sisukokkuvõtja",}]
  for i in range(20):
    ajalugu.append({"role": "user",
    "content": artiklid[i]})
    ajalugu.append({"role": "assistant",
    "content": kokkuvote[i]})
  return ajalugu

message_history = tekita_viipa_ajalugu()

In [ ]:
client = Groq(
    api_key=API_KEY,
)

In [ ]:
for i in range(1, 5):
  trucnated_history = message_history[:7]
  trucnated_history.append(
      {
          "role": "user",
          "content": read_file_into_memory(f"/content/drive/MyDrive/loputoo/testkorpus/uudised/uudis{i}/uudis{i}_alg.txt")
      }
  )

  chat_completion = client.chat.completions.create(
        messages=trucnated_history,
        model="llama-3.3-70b-versatile",
    )


  with open(f"/content/drive/MyDrive/loputoo/testkorpus/uudis{i}-llm.txt", "w") as file:
    file.write(chat_completion.choices[0].message.content)
  print(i)
  sleep(60)
